In [2]:
import numpy as np
import sympy as sym

In [3]:
T= sym.Symbol('T',ral=True)
TD = sym.Symbol('TD',ral=True)
Td=300
x = sym.Symbol('x',real=True)
y = sym.Symbol('y',real=True)

In [4]:
def GetLeg(n,x):

    if n==0:
        poly = sym.Number(1)
    elif n==1:
        poly = x
    else:
        poly = ((2*n-1)*x*GetLeg(n-1,x)-(n-1)*GetLeg(n-2,x))/n
   
    return poly

In [5]:
GetLeg(5,x)

-4*x*(3*x**2/2 - 1/2)/3 + 9*x*(-9*x**2/8 + 7*x*(5*x*(3*x**2/2 - 1/2)/3 - 2*x/3)/4 + 3/8)/5 + 8*x/15

In [6]:
def GetDLegendre(n,x):
    Pn = GetLeg(n,x)
    return sym.diff(Pn,x,1)

In [7]:
def GetNewton(f,df,xn,itmax=10000,precision=1e-14):
    
    error = 1.
    it = 0
    
    while error >= precision and it < itmax:
        
        try:
            
            xn1 = xn - f(xn)/df(xn)
            
            error = np.abs(f(xn)/df(xn))
            
        except ZeroDivisionError:
            print('Zero Division')
            
        xn = xn1
        it += 1
        
    if it == itmax:
        return False
    else:
        return xn

In [8]:
def GetRoots(f,df,x,tolerancia = 10):
    
    Roots = np.array([])
    
    for i in x:
        
        root = GetNewton(f,df,i)

        if  type(root)!=bool:
            croot = np.round( root, tolerancia )
            
            if croot not in Roots:
                Roots = np.append(Roots, croot)
                
    Roots.sort()
    
    return Roots

In [9]:
def GetAllRootsGLeg(n):

    xn = np.linspace(-1,1,100)
    
    Legendre = []
    DLegendre = []
    
    for i in range(n+1):
        Legendre.append(GetLeg(i,x))
        DLegendre.append(GetDLegendre(i,x))
    
    poly = sym.lambdify([x],Legendre[n],'numpy')
    Dpoly = sym.lambdify([x],DLegendre[n],'numpy')
    Roots = GetRoots(poly,Dpoly,xn)

    if len(Roots) != n:
        ValueError('El número de raíces debe ser igual al n del polinomio.')
    
    return Roots

In [10]:
raices = GetAllRootsGLeg(5)

In [11]:
def GetWeightsGLeg(n):

    Roots = GetAllRootsGLeg(n)

    

    DLegendre = []
    
    for i in range(n+1):
        DLegendre.append(GetDLegendre(i,x))
    
    Dpoly = sym.lambdify([x],DLegendre[n],'numpy')
    Weights = 2/((1-Roots**2)*Dpoly(Roots)**2)
    
    return Weights

In [12]:
pesos = GetWeightsGLeg(5)

In [13]:
T=np.linspace(1,20,200000)

In [23]:
funcion = lambda x,DT,T: (np.tanh(np.sqrt(((x**2)+(DT**2))*((150)/(T)))))/(np.sqrt((x**2)+(DT**2)))

In [24]:

for t in T:
    fu= lambda x: funcion(x,0.0001,t)
    I = 0
    for i in range(5):
        I += pesos[i]*fu(raices[i])
    I=I/2
    #print (I)
    if np.abs(I-(1/(0.3))) < 0.0004:
        print (t)

2.5447077235386177
2.5448027240136204
2.5448977244886226
2.544992724963625
2.545087725438627
2.5451827259136297
2.545277726388632
2.5453727268636346
2.545467727338637
2.545562727813639
2.5456577282886412
2.545752728763644
2.5458477292386466
2.5459427297136488
2.546037730188651
2.546132730663653
2.546227731138656
2.546322731613658
2.5464177320886607
2.546512732563663
